In [1]:
!pip install tiktoken
import torch
import numpy as np
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer, DistilBertModel # here distil refers to small model

from tqdm import tqdm
import argparse
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tiktoken

  Using cached tiktoken-0.12.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (6.7 kB)
Using cached tiktoken-0.12.0-cp312-cp312-manylinux_2_28_x86_64.whl (1.2 MB)


2025-12-18 18:26:54.681369: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
s3_path = 's3://sagemaker-text-classifier-uci-news-aggregation/training_data/newsCorpora.csv'
df = pd.read_csv(s3_path, sep='\t', names=['ID', 'TITLE', 'URL',' PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP'])

df = df[['TITLE','CATEGORY']]
my_dict = {

    'e': 'Entertainment',
    'b': 'Business',
    't': 'Science',
    'm': 'Health'
}    

In [3]:




def update_catagory(x):
    return my_dict[x]

# This calls the function ONLY if 'x' is in the dictionary.
# Otherwise, it skips the function call and keeps 'x' as is.
df['CATEGORY'] = df['CATEGORY'].apply(lambda x: update_catagory(x) if x in my_dict else x)

df





,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",Business
1,Fed's Charles Plosser sees high bar for change...,Business
2,US open: Stocks fall after Fed official hints ...,Business
3,"Fed risks falling 'behind the curve', Charles ...",Business
4,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,Business
...,...,...
422414,Surgeons to remove 4-year-old's rib to rebuild...,Health
422415,Boy to have surgery on esophagus after battery...,Health
422416,Child who swallowed battery to have reconstruc...,Health
422417,Phoenix boy undergoes surgery to repair throat...,Health


In [4]:
#sometimes after big model training due to issue of s3 bucket it can fails which leads to time loss and resource loss, its mendatory to check with small instance first
# to do so
df = df.sample(frac=0.05, random_state=1)
df = df.reset_index(drop=True)
#Tips end here
print(df)

                                                   TITLE       CATEGORY
0                 Murdoch's bid for Time Warner rejected       Business
1      Rescuers close in on 3 trapped Honduran miners...       Business
2      Johnny Depp - Johnny Depp Served With Legal Pa...  Entertainment
3      Apple prepping move into "smart home" connecti...        Science
4      Ripped First Look: Dwayne Johnson as Brett Rat...  Entertainment
...                                                  ...            ...
21116  Fed Beige Book: Activity, labor markets improv...       Business
21117                           National Agriculture Day       Business
21118  Placenta Home to Diverse Bacteria That May Aff...         Health
21119  US TV network Fox to air live 'Grease' musical...  Entertainment
21120    Elton John Is Getting Married! Details Revealed  Entertainment

[21121 rows x 2 columns]


In [5]:
encode_dict = {}

def encode_cat(x):
    if x not in encode_dict.keys():
        encode_dict[x] = len(encode_dict)

    return encode_dict[x]

In [6]:
df['ENCODE_CAT'] = df['CATEGORY'].apply(lambda x: encode_cat(x))

In [7]:
df

,TITLE,CATEGORY,ENCODE_CAT
0,Murdoch's bid for Time Warner rejected,Business,0
1,Rescuers close in on 3 trapped Honduran miners...,Business,0
2,Johnny Depp - Johnny Depp Served With Legal Pa...,Entertainment,1
3,"Apple prepping move into ""smart home"" connecti...",Science,2
4,Ripped First Look: Dwayne Johnson as Brett Rat...,Entertainment,1
...,...,...,...
21116,"Fed Beige Book: Activity, labor markets improv...",Business,0
21117,National Agriculture Day,Business,0
21118,Placenta Home to Diverse Bacteria That May Aff...,Health,3
21119,US TV network Fox to air live 'Grease' musical...,Entertainment,1


In [8]:
pip show transformers

Name: transformers
Version: 4.57.1
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /opt/conda/lib/python3.12/site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: amazon_sagemaker_jupyter_ai_q_developer, autogluon.multimodal, autogluon.timeseries
Note: you may need to restart the kernel to use updated packages.


In [27]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
inputs = tokenizer.encode_plus( " I love Cricket very much specially at the time of vaction when all of my frieds are available and they like to meet together",
                               "I love chess",#working alike -> [CLS] Sentence A [SEP] Sentence B [SEP]
                               add_special_tokens = True,#Automatically adds the [CLS] at the start and [SEP] at the appropriate places.
                               max_length = 40, #as the data set is have only title it is ok for nlp task it need to be high,Sets the limit. The final output must be exactly 40 tokens long.
                               padding = 'max_length',#If the combined length is less than 40 (which is likely here), it adds "Padding Tokens" (zeros) to the end until the length hits 40.
                               truncation = True, #zIf the combined length (Sentence A + Sentence B + 3 special tokens) is more than 40, it will cut off words from the end to prevent an error.
                               return_token_type_ids= True,#Asks the tokenizer to create a "segment ID" mask.Sentence A gets ID 0.Sentence B gets ID 1.DistilBERT typically doesn't use these (it ignores them or treats them as zeros), but BERT uses them heavily to differentiate sentences.
                               return_attention_mask = True
                               

                             )
print("inpput ids: ",inputs['input_ids']) # This is the list of integers representing the words.
print("Attention maks: ", inputs['attention_mask'])# attention_mask->It stops the model from trying to find meaning in the empty padding zeros.
print("Token type Ids:", inputs['token_type_ids'])#This is meant to help the model distinguish between Sentence A and Sentence B

Keyword arguments {'add_spacial_tockens': True} not recognized.
Keyword arguments {'add_spacial_tockens': True} not recognized.


inpput ids:  [101, 1045, 2293, 4533, 2200, 2172, 11974, 2012, 1996, 2051, 1997, 12436, 7542, 2043, 2035, 1997, 2026, 13017, 2015, 2024, 2800, 1998, 2027, 2066, 2000, 3113, 2362, 102, 1045, 2293, 7433, 102, 0, 0, 0, 0, 0, 0, 0, 0]
Attention maks:  [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Token type Ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
